In [10]:
import os
import pandas as pd

from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# ===== 0) ファイルがColabに存在するか確認 =====
files = os.listdir("/content")

file_2022 = "2022_年間売上表.xlsx"
file_2023 = "2023_年間売上表.xlsx"

if file_2022 not in files:
    raise FileNotFoundError(f"'{file_2022}' が /content に見つかりません。アップロード名を確認してください。")
if file_2023 not in files:
    raise FileNotFoundError(f"'{file_2023}' が /content に見つかりません。アップロード名を確認してください。")

# ===== 1) Excel読み込み（まずはシート名を確認） =====
xlsx_2022 = pd.ExcelFile(file_2022)
xlsx_2023 = pd.ExcelFile(file_2023)

df_2022 = pd.read_excel(file_2022, sheet_name=sheet_2022)
df_2023 = pd.read_excel(file_2023, sheet_name=sheet_2023)

# ===== 2) 列名のズレ対策（余計な空白・改行を除去） =====
df_2022.columns = df_2022.columns.astype(str).str.strip()
df_2023.columns = df_2023.columns.astype(str).str.strip()

required_cols = {"売上年", "商品", "金額（千円）"}
if not required_cols.issubset(set(df_2022.columns)):
    raise ValueError(f"2022側の列名が要件と違います。現在の列: {list(df_2022.columns)}")
if not required_cols.issubset(set(df_2023.columns)):
    raise ValueError(f"2023側の列名が要件と違います。現在の列: {list(df_2023.columns)}")

# ===== 3) 連結 → 商品×売上年で合計 =====
df_all = pd.concat([df_2022, df_2023], ignore_index=True)

# 金額が文字列になってる場合に備えて数値化
df_all["金額（千円）"] = pd.to_numeric(df_all["金額（千円）"], errors="raise")

df_summary = (
    df_all.groupby(["商品", "売上年"], as_index=False)["金額（千円）"]
    .sum()
    .sort_values(["売上年", "商品"])
    .reset_index(drop=True)
)

# ===== 4) Excelへ書き込み =====
output_file = "売上集計表.xlsx"
sheet_name_out = "売上集計表"

df_summary.to_excel(output_file, index=False, sheet_name=sheet_name_out)

# ===== 5) openpyxlでヘッダー色 =====
wb = load_workbook(output_file)
ws = wb[sheet_name_out]

fill = PatternFill(fill_type="solid", start_color="F2F2F2", end_color="F2F2F2")

for cell in ws[1]:
    cell.fill = fill

wb.save(output_file)
print("🎉 作成完了:", output_file)


🎉 作成完了: 売上集計表.xlsx
